# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel, paired_cosine_distances, cosine_similarity
from nltk.corpus import stopwords
import nltk
import streamlit as st
import shap
import streamlit.components.v1 as stc
import pickle

In [3]:
merge_dataset = pd.read_csv("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Data/merged_description.csv", index_col= 0)

In [25]:
merge_dataset

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,average_playtime,median_playtime,owners,price,total_ratings,percent_positive_ratings,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,17612,317,10000000-20000000,7.19,127873,0.973888,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,277,62,5000000-10000000,3.99,3951,0.839787,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,0,187,34,5000000-10000000,3.99,3814,0.895648,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,258,184,5000000-10000000,3.99,1540,0.826623,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,0,624,415,5000000-10000000,3.99,5538,0.947996,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,...,7,0,0,0-20000,2.09,3,1.000000,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",The Room of Pandora is a third-person interact...
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,...,0,0,0,0-20000,1.69,9,0.888889,Have you ever been so lonely that no one but y...,Have you ever been so lonely that no one but y...,Cyber Gun is a hardcore first-person shooter w...
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,...,24,0,0,0-20000,3.99,1,0.000000,<strong>Super Star Blast </strong>is a space b...,<strong>Super Star Blast </strong>is a space b...,Super Star Blast is a space based game with ch...
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,...,0,0,0,0-20000,5.19,2,1.000000,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...


## Load TFIDF

In [4]:
tfidf_matrix = pickle.load(open("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Code/tfidf_matrix.pkl", 'rb'))

In [21]:
# tfidf_matrix.shape

# Model

In [6]:
def content_recommender(name, games, similarities, vote_threshold=1000, rating_threshold=0.7) :
    
    # Get the game by the title
    game_index = games[games['name']==name].index
    
    # Create a dataframe with the game id, name, and rating information with similarity
    sim_df = pd.DataFrame(
        {'appid': games['appid'],
         'game': games['name'], 
         'similarity': np.array(similarities[game_index, :].todense()).squeeze(),
         'diversity': 1- np.array(similarities[game_index, :].todense()).squeeze(),
         'vote_count': games['total_ratings'],
         'percent_positive_ratings': games['percent_positive_ratings']
        })
    
    # Get the top 10 games that satisfy our thresholds
    top_games = sim_df[(sim_df['vote_count']>vote_threshold) & 
                       (sim_df['percent_positive_ratings']>rating_threshold)].sort_values(by='similarity', ascending=False).head(10)
    
    return top_games

In [7]:
# # find index of each name to bring the similar game by entring game name
# indexs = pd.Series(merge_dataset.index, index = merge_dataset.name).drop_duplicates()

In [8]:
# # test recommendtion system based on sigmoid_kernel
# def recomend_game(name, num ,sig=sig):
#     indx = indexs[name]
#     sig_score = list(enumerate(sig[indx])) # get similarity score
#     sig_score = sorted(sig_score, key= lambda x:x[1], reverse = True) # sort game
#     sig_score = sig_score[1:num+1]
    
#     game_index = [i[0] for i in sig_score] # get index of each score
    
#     return merge_dataset.name.iloc[game_index] # return the name


### 1) sigmoid_kernel

In [11]:
# # to find  content based similarity  will try to use sigmoid kernel to find the propabilty of detailed_description similarity
# sig = sigmoid_kernel(tfidf_matrix,tfidf_matrix)
# # to much time but works 


In [27]:
# recomend_game("PAYDAY 2",3)

### 2) cosine_similarity

In [10]:
import _pickle as cPickle
# import cPickle as pickle
import gc

f = open("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Code/cosine_similarity_model.pkl", 'rb')

# disable garbage collector
gc.disable()

similarities = cPickle.load(f)

# enable garbage collector again
gc.enable()
f.close()

In [11]:
# similarities = pickle.load(open("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Code/cosine_similarity_model.pkl", 'rb'))



In [13]:
# similarities


In [16]:
# similar_games = content_recommender('PAYDAY 2', merge_dataset, similarities, 
#                                     rating_threshold=0.80)
# similar_games.head(5).sort_values(by='percent_positive_ratings',ascending=False)

# web streamlite

In [20]:

def main():
    st.title("Game Recommendation System")
    menu = ["Collaborative Recommendation","Content-based Recommendation","Others"]
    choice = st.sidebar.selectbox("Menu",menu)
    
    #df = load_data("data/udemy_course_data.csv")
    
    if choice == "Collaborative Recommendation":
        st.subheader("Recommendation based on users preferences")
        #st.dataframe(df.head(10))
        
    elif choice == "Content-based Recommendation":
        st.subheader("Recommendation based on games description")
        #cosine_sim_mat = vectorize_text_to_cosine_mat(df['name'])
        search_term = st.text_input("Search")
        num_of_rec = st.sidebar.number_input("Number",4,30,7)
        if st.button("Recommend"):
            if search_term is not None:
                try:
                    
                    #results = recomend_game(search_term,num_of_rec)
                    results = similar_games = content_recommender(search_term, merge_dataset, similarities, 
                                    rating_threshold=0.80).sort_values(by='percent_positive_ratings',ascending=False)
#                     similar_games.head(5).sort_values(by='percent_positive_ratings',ascending=False)
                    st.write(results)
                    with st.beta_expander("Results as JSON"):
                        results_json = results.to_dict('index')
                        st.write(results_json)
                
                except:
                    results= "Not Found"
                    st.warning(results)
                    st.info("Suggested Options include")
    else:
        st.subheader("Others")
        st.text("Game Recommandation System is a ....")
        


if __name__ == '__main__':
    main()